### 10 Folium/Altair Maps for Geospatial Analytics in Minutes

Practical, copy-paste map recipes that turn raw lat/longs and polygons into decisions — without becoming a GIS expert.

In [1]:
# %pip install pandas numpy

In [2]:
# Setup & Tiny Dataset (once)

import pandas as pd
import numpy as np

# 1) Random points roughly around Bengaluru
rng = np.random.default_rng(42)
N = 1000
df_points = pd.DataFrame({
    "lat": 12.9 + rng.normal(0, 0.12, N),
    "lon": 77.6 + rng.normal(0, 0.12, N),
    "value": rng.integers(1, 100, N),
    "category": rng.choice(["A","B","C"], N, p=[0.5,0.3,0.2])
})

# 2) Minimal GeoJSON-like polygon (pretend districts; keep small)
districts = {
  "type":"FeatureCollection",
  "features":[
    {"type":"Feature","properties":{"name":"North"},"geometry":{"type":"Polygon","coordinates":[[[77.45,13.02],[77.9,13.02],[77.9,13.25],[77.45,13.25],[77.45,13.02]]]}},
    {"type":"Feature","properties":{"name":"South"},"geometry":{"type":"Polygon","coordinates":[[[77.45,12.55],[77.9,12.55],[77.9,12.9],[77.45,12.9],[77.45,12.55]]]}},
  ]
}

#### Folium: interactive analysis you can hand to anyone
1) Base map + smart marker clusters
When: Quick exploration across thousands of points without drowning in pins.



In [1]:
# %pip install folium

In [3]:
# %pip install folium

import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[12.97, 77.59], zoom_start=11, tiles="CartoDB positron")
cluster = MarkerCluster().add_to(m)

for _, r in df_points.sample(500).iterrows():
    folium.Marker(
        location=[r.lat, r.lon],
        popup=f"value: {r.value}, cat: {r.category}"
    ).add_to(cluster)

m


#### 2) Heatmap for density hot zones
When: You want where-are-people-concentrated without worrying about bins.

In [5]:
from folium.plugins import HeatMap

m = folium.Map(location=[12.97, 77.59], zoom_start=11, tiles="CartoDB dark_matter")
HeatMap(df_points[["lat","lon","value"]].values.tolist(), radius=18, blur=15).add_to(m)
m

### 3) Choropleth over simple polygons
When: Region-level rollups (districts, sales territories) tell the real story.

In [6]:
import json
gjson = json.loads(json.dumps(districts))

# fake regional metric
region_df = pd.DataFrame({"name":["North","South"], "score":[68, 45]})

m = folium.Map(location=[12.97,77.59], zoom_start=10, tiles="CartoDB positron")
folium.Choropleth(
    geo_data=gjson,
    data=region_df,
    columns=("name","score"),
    key_on="feature.properties.name",
    fill_color="YlOrRd",
    fill_opacity=0.75,
    line_opacity=0.6,
    legend_name="Score"
).add_to(m)
m

#### 4) Category-styled circle markers with layer control
When: Compare patterns by segment without making separate maps.

In [ ]:
from folium import FeatureGroup, LayerControl

m = folium.Map(location=[12.97, 77.59], zoom_start=11)
colors = {"A": "#2ecc71", "B": "#e67e22", "C": "#3498db"}

for cat, group in df_points.groupby("category"):
    fg = FeatureGroup(name=f"Category {cat}", show=True).add_to(m)
    # Dynamically adjust sample size
    sample_size = min(250, len(group))
    for _, r in group.sample(sample_size).iterrows():
        folium.CircleMarker(
            [r.lat, r.lon], radius=5, color=colors[cat], fill=True, fill_opacity=0.6
        ).add_to(fg)

LayerControl(collapsed=False).add_to(m)
m

ValueError: Cannot take a larger sample than population when 'replace=False'